# --- Day 12 Hill Climbing Algorithm ---

https://adventofcode.com/2022/day/12

In [1]:
from collections import defaultdict
from string import ascii_letters

## Get Input Data

In [2]:
def get_data(filename):
    heightmap = []
    with open(f'../inputs/{filename}.txt') as _file:
        for line in _file:
            heightmap.append(line.rstrip())
    return heightmap

In [3]:
test_heightmap = get_data('test_heightmap')
test_heightmap

['Sabqponm', 'abcryxxl', 'accszExk', 'acctuvwj', 'abdefghi']

In [4]:
heightmap = get_data('heightmap')
heightmap[:5]

['abcccccccccaaaaaaaaaaccccccccccccaaaaaaaaccaaccccccccccccccccccccccccccccccccccccccccccccaaaaaa',
 'abccccccccccaaaaaaaaaccccccccccccaaaaaaaaaaaacccccccccccaacccacccccccccccccccccccccccccccaaaaaa',
 'abcccccccccccaaaaaaacccccccccccccaaaaaaaaaaaaaacccccccccaaacaacccccccccaaaccccccccccccccccaaaaa',
 'abccccccccccaaaaaaccccccccccccccaaaaaaaaaaaaaaaccccccccccaaaaaccccccccccaaacccccccccccccccccaaa',
 'abccccccccccaaaaaaaccccccccccccaaaaaaaaaaaaaacccccccccccaaaaaacccccccccaaaacccccccccccccccccaac']

In [5]:
def build_graph(heightmap):
    """Build an undirected, unweighted graph based on the raw heightmap data, where
    every letter corresponds to an elevation level, "S" is the starting node, and "E" 
    is the ending node and you can only move up one level at a time, at most.
    
    Parameters
    ----------
    heightmap : list
        Each item is a string of characters, representing a row in the raw heightmap

    Returns
    -------
    graph, start_pos, end_pos : dictionary, tuple, tuple
        The graph will have (row, col) tuples as keys representing a node and each value
        will be a list of permissable moves to a next node.
    """

    graph = {}

    row_max = len(heightmap) - 1
    col_max = len(heightmap[0]) -1

    for row in range(len(heightmap)):
        if 'S' in heightmap[row]:
            start_pos = (row, heightmap[row].index('S'))
            heightmap[row] = heightmap[row].replace('S', 'a')
        if 'E' in heightmap[row]:
            end_pos = (row, heightmap[row].index('E'))
            heightmap[row] = heightmap[row].replace('E', 'z')

        for col in range(len(heightmap[0])):
            current_height = ascii_letters.index(heightmap[row][col])
    
            graph[(row, col)] = []

            for deltas in [(1, 0), (0, 1), (-1, 0), (0, -1)]:
                row_delta, col_delta = deltas
                if (0 <= row + row_delta <= row_max) and (0 <= col + col_delta <= col_max):
                    delta_height = ascii_letters.index(heightmap[row + row_delta][col + col_delta])
                    if delta_height - current_height <= 1:
                        graph[(row, col)].append((row + row_delta, col + col_delta))

    return graph, start_pos, end_pos

In [6]:
test_graph, start_pos, end_pos = build_graph(get_data('test_heightmap'))
print(start_pos, end_pos)
test_graph

(0, 0) (2, 5)


{(0, 0): [(1, 0), (0, 1)],
 (0, 1): [(1, 1), (0, 2), (0, 0)],
 (0, 2): [(1, 2), (0, 1)],
 (0, 3): [(1, 3), (0, 4), (0, 2)],
 (0, 4): [(0, 5), (0, 3)],
 (0, 5): [(0, 6), (0, 4)],
 (0, 6): [(0, 7), (0, 5)],
 (0, 7): [(1, 7), (0, 6)],
 (1, 0): [(2, 0), (1, 1), (0, 0)],
 (1, 1): [(2, 1), (1, 2), (0, 1), (1, 0)],
 (1, 2): [(2, 2), (0, 2), (1, 1)],
 (1, 3): [(2, 3), (0, 3), (1, 2)],
 (1, 4): [(2, 4), (1, 5), (0, 4), (1, 3)],
 (1, 5): [(1, 6), (0, 5), (1, 4)],
 (1, 6): [(2, 6), (1, 7), (0, 6), (1, 5)],
 (1, 7): [(2, 7), (0, 7)],
 (2, 0): [(3, 0), (1, 0)],
 (2, 1): [(3, 1), (2, 2), (1, 1), (2, 0)],
 (2, 2): [(3, 2), (1, 2), (2, 1)],
 (2, 3): [(3, 3), (1, 3), (2, 2)],
 (2, 4): [(3, 4), (2, 5), (1, 4), (2, 3)],
 (2, 5): [(3, 5), (2, 6), (1, 5), (2, 4)],
 (2, 6): [(3, 6), (2, 7), (1, 6)],
 (2, 7): [(3, 7), (1, 7)],
 (3, 0): [(4, 0), (2, 0)],
 (3, 1): [(4, 1), (3, 2), (2, 1), (3, 0)],
 (3, 2): [(4, 2), (2, 2), (3, 1)],
 (3, 3): [(4, 3), (3, 4), (2, 3), (3, 2)],
 (3, 4): [(4, 4), (3, 5), (3, 3)],
 

## Part 1
---

In [7]:
def find_shortest_path(graph, start_node, end_node):
    """Implementation of breadth-first search algorithm, which will find the shortest path
    between two nodes in an undirected, unweighted graph.
    
    Slight modifications and added comments from code found here:
    https://www.python.org/doc/essays/graphs/, by Eryk Kopczyński

    Parameters
    ----------
    graph : dict
        An undirected, unweighted graph. Each key is a node, and each value is a list of
        nodes connected to the current node by an edge.
    start_node : tuple
        Starting node, as a (row, col) tuple.
    end_node : tuple
        Ending node, as a (row, col) tuple.

    Returns
    -------
    list
        List containing the shortest path from start to end nodes.
    """

    # Dictionary to hold the breadth-first path to each node *from the starting node*
    # Each key will be a node, and each value will contain the list of nodes in the path.
    paths = {start_node : [start_node]}
    
    # Use a double ended que ("deque") to keep track of which nodes to check out next
    deque = [start_node]

    # Keep processing until all the nodes have been visited, or until we get to the end node,
    # which is when the function returns.
    while len(deque) > 0:

        current_node = deque.pop(0)

        for next_node in graph[current_node]:
            if next_node not in paths:
                paths[next_node] = paths[current_node] + [next_node]
                deque.append(next_node)

            # When we get to the end node, the path there will always be the shortest possible
            # path because we are doing a breadth-first search (instead of a depth-first search)
            if next_node == end_node:
                return paths[end_node]

## Run on Test Data

In [8]:
# Subtract 1 from the length, because we are interested in counting the steps (edges), not the nodes
len(find_shortest_path(*build_graph(get_data('test_heightmap')))) - 1 == 31

True

## Run on Input Data

In [9]:
len(find_shortest_path(*build_graph(get_data('heightmap')))) - 1

420

## Part 2
---

In [10]:
def build_graph2(heightmap):
    """Build an undirected, unweighted graph based on the raw heightmap data, where
    every letter corresponds to an elevation level, "S" is the starting node, and "E" 
    is the ending node and you can only move up one level at a time, at most.
    
    Parameters
    ----------
    heightmap : list
        Each item is a string of characters, representing a row in the raw heightmap

    Returns
    -------
    graph, start_pos, end_pos : dictionary, tuple, tuple
        The graph will have (row, col) tuples as keys representing a node and each value
        will be a list of permissable moves to a next node.

        In part 2, start_pos will be a list of nodes that are equal to 'a', or 0.
    """
    
    graph = {}
    start_pos = []

    row_max = len(heightmap) - 1
    col_max = len(heightmap[0]) -1

    for row in range(len(heightmap)):
        if 'S' in heightmap[row]:
            # start_pos = (row, heightmap[row].index('S'))  -- From Part 1
            heightmap[row] = heightmap[row].replace('S', 'a')
        if 'E' in heightmap[row]:
            end_pos = (row, heightmap[row].index('E'))
            heightmap[row] = heightmap[row].replace('E', 'z')

        for col in range(len(heightmap[0])):
            current_height = ascii_letters.index(heightmap[row][col])
            
            # Compile list of nodes == 'a' (or 0 after conversion)  -- For Part 2
            if current_height == 0:
                start_pos.append((row, col))

            graph[(row, col)] = []

            for deltas in [(1, 0), (0, 1), (-1, 0), (0, -1)]:
                row_delta, col_delta = deltas
                if (0 <= row + row_delta <= row_max) and (0 <= col + col_delta <= col_max):
                    delta_height = ascii_letters.index(heightmap[row + row_delta][col + col_delta])
                    if delta_height - current_height <= 1:
                        graph[(row, col)].append((row + row_delta, col + col_delta))

    return graph, start_pos, end_pos

In [11]:
def solve_part2(filename):
    """Find the shortest path to the end node of a graph, but start from any node
    that is equal to 'a'.
    
    Parameters
    ----------
    filename : str
        Name of file with raw heightmap data.

    Returns
    -------
    int
    """

    len_paths_from_a = []
    graph, a_nodes, end_node = build_graph2(get_data(filename))
    for node in a_nodes:
        if find_shortest_path(graph, node, end_node) != None:
            len_paths_from_a.append(len(find_shortest_path(graph, node, end_node)) - 1)

    return min(len_paths_from_a)

### Run on Test Data

In [12]:
solve_part2('test_heightmap') == 29

True

### Run on Input Data

In [13]:
solve_part2('heightmap')

414